# Exercise 04 - Due Friday, October 21 at 12pm

*Objectives*: Gain experience loading a CSV dataset into a database model you define yourself and using SQL to explore its contents. Explore the data by writing and executing a number of SQL queries using common syntax and functions and describing your findings.

*Grading criteria*: The tasks should all be completed, and questions should all be answered with SQL queries in the space provided, unless a text answer is requested. Results should be correct, according to the embedded tests. The notebook itself should be completely reproducible; from start to finish, another person should be able to use the same code to obtain the same results as yours.  Note that you will receive no more than partial credit if you do not add text/markdown cells explaining your thinking at each major step in each problem.

*Deadline*: Friday, October 21, 12pm. Submit your notebook to Blackboard and push it to your GitHub repository.

## Setup - obtain data and create database

The US City Open Data Census has data on [restaurant inspections from all over the country](http://us-city.census.okfn.org/dataset/food-safety).  Let's take a look at data from Albuquerque.  *Note*: metadata and other details for this dataset are available at http://data.cabq.gov/business/foodinspections/.

Fyi, to save you a step, I've converted this file into a clean CSV file by using the `dos2unix` command to change its line endings and to switch it to use commas instead of tabs using `csvformat -t inputfile.csv > outputfile.csv`.

First we download the dataset:

In [1]:
!wget https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/exercises/abq-food-inspections-fy2010.csv

--2016-10-24 01:13:08--  https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/exercises/abq-food-inspections-fy2010.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.32.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.32.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5025226 (4.8M) [text/plain]
Saving to: ‘abq-food-inspections-fy2010.csv’

abq-food-inspection 100%[=====================>]   4.79M  --.-KB/s   in 0.09s  

2016-10-24 01:13:08 (51.4 MB/s) - ‘abq-food-inspections-fy2010.csv’ saved [5025226/5025226]



## Problem 1 - examine the data (35 points)

Use `csvstat` and other tools as you see fit and observe its contents.  Review the metadata definitions linked to above.  Describe what you see:  What do all the columns mean?  Are there null values?  Which columns interest you the most?  Which columns present some complications or questions you would like to ask?

Insert a combination of code and text/markdown cells here to review the data and describe what you find.

Step 1: Rename csv file into something shorter so it is easier to reference

In [2]:
!mv abq-food-inspections-fy2010.csv abq.csv

Step 2: Examine columns

In [3]:
!csvcut -n abq.csv

  1: FACILITY_NAME
  2: FACILITY_KEY
  3: SITE_ADDRESS
  4: CITY
  5: STATE
  6: ZIP
  7: OWNER_KEY
  8: OWNER_NAME
  9: NATURE_OF_BUSINESS
 10: STREET_NUMBER
 11: STREET_NAME
 12: STREET_TYPE
 13: POST_DIRECTIONAL
 14: PHONE
 15: PROGRAM_CATEGORY
 16: PROGRAM_CATEGORY_DESCRIPTION
 17: INSPECTION_DATE
 18: INSPECTION_TYPE
 19: INSPECTION_DESC
 20: SERIAL_NUM
 21: ACTION_CODE
 22: ACTION_DESC
 23: RESULT_CODE
 24: RESULT_DESC
 25: VIOLATION_CODE
 26: VIOLATION_DESC
 27: INSPECTION_MEMO


The columns appear to contain a variety of fields pertaining to information about the restaurant location, the owner, the nature of the inspection, and the outcome of the inspection. The columns that interest me the most would be violation_code and violation_desc. As someone who eats out regularly, I would want to make sure to avoid these restaurants should I ever visit Albuquerque!

Step 3: Examine sample of data (broken into 3 steps due to width of table)

In [4]:
!csvcut -c1-9 abq.csv | head -10 | csvlook

|--------------------------+--------------+-------------------------------+-------------+-------+-------+-----------+-------------------------+---------------------|
|  FACILITY_NAME           | FACILITY_KEY | SITE_ADDRESS                  | CITY        | STATE | ZIP   | OWNER_KEY | OWNER_NAME              | NATURE_OF_BUSINESS  |
|--------------------------+--------------+-------------------------------+-------------+-------+-------+-----------+-------------------------+---------------------|
|  00BSAY IT WITH CAKE     | 50534        | 10200 GOLF COURSE RD NW STE B | ALBUQUERQUE | NM    | 87114 | 47341     | OOBSAY IT WITH CAKE     | CAKE DECORATING     |
|  00BSAY IT WITH CAKE     | 50534        | 10200 GOLF COURSE RD NW STE B | ALBUQUERQUE | NM    | 87114 | 47341     | OOBSAY IT WITH CAKE     | CAKE DECORATING     |
|  00BSAY IT WITH CAKE     | 50534        | 10200 GOLF COURSE RD NW STE B | ALBUQUERQUE | NM    | 87114 | 47341     | OOBSAY IT WITH CAKE     | CAKE DECORATING     |
|  0

In [5]:
!csvcut -c10-18 abq.csv | head -10 | csvlook

|----------------+-------------+-------------+------------------+------------+------------------+------------------------------+---------------------+------------------|
|  STREET_NUMBER | STREET_NAME | STREET_TYPE | POST_DIRECTIONAL | PHONE      | PROGRAM_CATEGORY | PROGRAM_CATEGORY_DESCRIPTION | INSPECTION_DATE     | INSPECTION_TYPE  |
|----------------+-------------+-------------+------------------+------------+------------------+------------------------------+---------------------+------------------|
|  10200         | GOLF COURSE | RD          | NW               | 5058339563 | 0405             | Food Processor               | 2009-10-21 00:00:00 |                  |
|  10200         | GOLF COURSE | RD          | NW               | 5058339563 | 0405             | Food Processor               | 2009-10-21 00:00:00 |                  |
|  10200         | GOLF COURSE | RD          | NW               | 5058339563 | 0405             | Food Processor               | 2009-10-21 00:00:00 |

In [6]:
!csvcut -c19-27 abq.csv | head -10 | csvlook

|--------------------------+------------+-------------+-------------+-------------+----------------+----------------+----------------------------------+------------------|
|  INSPECTION_DESC         | SERIAL_NUM | ACTION_CODE | ACTION_DESC | RESULT_CODE | RESULT_DESC    | VIOLATION_CODE | VIOLATION_DESC                   | INSPECTION_MEMO  |
|--------------------------+------------+-------------+-------------+-------------+----------------+----------------+----------------------------------+------------------|
|  EMERGENCY RESPONSE      | DA8009125  | 00          | NON-GRADED  | 00          | NOT APPLICABLE |     68         | Delivered Informational Brochure |                  |
|  ROUTINE FOOD INSPECTION | DA8009123  | 11          | APPROVED    | 03          | IN COMPLIANCE  |     00         | No Violations Found              |                  |
|  ROUTINE FOOD INSPECTION | DA8009123  | 11          | APPROVED    | 03          | IN COMPLIANCE  |     55         | Additional Comments   

Step 4: View summary statistics for each field

In [7]:
!csvcut abq.csv | csvstat

  1. FACILITY_NAME
	<class 'str'>
	Nulls: False
	Unique values: 2615
	5 most frequent values:
		SMITHS FOOD AND DRUG CENTERS INC:	154
		ISOTOPES PARK - OVATIONS:	93
		PER EHSII CECELIA GARCIA OOB LA REYNA DEL SUR:	85
		ECHO INC:	68
		ISLETA AMPHITHEATER:	68
	Max length: 77
  2. FACILITY_KEY
	<class 'int'>
	Nulls: False
	Min: 13
	Max: 101482
	Sum: 820405773
	Mean: 46730.79135338346
	Median: 50866.0
	Standard Deviation: 32659.794578837962
	Unique values: 2835
	5 most frequent values:
		46323:	93
		65863:	85
		80949:	68
		32817:	68
		91825:	66
  3. SITE_ADDRESS
	<class 'str'>
	Nulls: True
	Unique values: 2456
	5 most frequent values:
		1601 AVENIDA CESAR CHAVEZ SE:	94
		644 OLD COORS BLVD SW:	85
		8601 CENTRAL AV NE:	77
		2200 SUNPORT BLVD SE:	75
		5210 CENTRAL AV SE:	73
	Max length: 35
  4. CITY
	<class 'str'>
	Nulls: False
	Values: ALBUQUERQUE
  5. STATE
	<class 'str'>
	Nulls: False
	Values: NM, 87
  6. ZIP
	<class 'str'>
	Nulls: False
	Unique values: 48
	5 most frequent values:
		87110

My observations after looking at above stats are described below:

Null values: Null values exist in a handful of the fields including: site_address, nature_of_business, street_number, street_name, street_type, post_directional, phone, program_category_description, inspection_type, violation_code, violation_desc, and inspection_memo.

Potential questions/quirks:   
1) Why is the street address unavailable for certain restaurants eventhough city, state, and zip information is available?   
2) Why is nature_of_business null for some of these restaurants?   
3) Why are there null values in program_category_description but no null values in program_category? I would expect that every program_category value would have a value populated in program_category_description.   
4) Why are there no values in the inspection_type and inspection_memo fields?   
5) Why is there not a 1:1 relationship between facility_name and facility_key?

## Problem 2 - define a database model and load the data (35 points)

Based on what you found above, create and connect to a new database, define a database table in it, and load this dataset into it.  You may use either of the methods for this step you have seen in earlier class notebooks.  You may choose to eliminate variables/columns if they are not relevant or interesting to you - explain your reasoning if you do.  Either way, you should load a majority of the columns present in the source dataset into the database and all of its rows.

Once your data has loaded successfully, run a `COUNT(*)` query to verify that all the data has loaded correctly.

Insert a combination of code and text/markdown cells here to connect to your database, define your table, load its data, and verify the loaded records.

Step 1: Set up and configure database

In [8]:
%load_ext sql

/opt/conda/lib/python3.5/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/opt/conda/lib/python3.5/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [9]:
!echo 'redspot' | sudo -S service postgresql restart

[sudo] password for jovyan: Restarting PostgreSQL 9.5 database server: main.


In [10]:
!createdb -U dbuser week7

In [11]:
%sql postgresql://dbuser@localhost:5432/week7

'Connected: dbuser@week7'

Step 2: Create database. Field lengths derived from metadata document.

In [13]:
%%sql
DROP TABLE IF EXISTS abq;
CREATE TABLE abq (
    facility_name VARCHAR(202),
    facility_key VARCHAR(20),
    site_address VARCHAR(102),
    city VARCHAR(52),
    state VARCHAR(6),
    zip VARCHAR(22),
    owner_key VARCHAR(26),
    owner_name VARCHAR(202),
    nature_of_business VARCHAR(25),
    street_number VARCHAR(16),
    street_name VARCHAR(52),
    street_type VARCHAR(10),
    post_directional VARCHAR(6),
    phone VARCHAR(42),
    program_category VARCHAR(10),
    program_category_description VARCHAR(92),
    inspection_date TIMESTAMP,
    inspection_type VARCHAR(4),
    inspection_desc VARCHAR(102),
    serial_num VARCHAR(20),
    action_code VARCHAR(6),
    action_desc VARCHAR(102),
    result_code VARCHAR(6),
    result_desc VARCHAR(102),
    violation_code VARCHAR(14),
    violation_desc VARCHAR(202),
    inspection_memo VARCHAR(502)
)


Done.
Done.


[]

Step 3: Copy data from csv file to newly created database and confirm all rows (17,556) were successfully loaded. 

In [14]:
!pwd

/home/jovyan/work


In [15]:
%%sql
COPY abq FROM '/home/jovyan/work/abq.csv'
CSV
HEADER
QUOTE '"'
DELIMITER ',';

17556 rows affected.


[]

In [16]:
%%sql
SELECT COUNT(*) FROM abq;

1 rows affected.


count
17556


## Problem 3 - explore your data (30 points)

Now that the data is loaded, ask some questions of it!  Identify key variables of interest and note their ranges along with other useful descriptive statistics.  Choose and define a few lines of inquiry, execute queries for each, and describe what you find in your result sets.  Use any query techniques we've seen in class, including aggregate functions, transformations, subqueries, or others as appropriate. 

If you find interesting patterns, adding plots to your exploration would be useful.

Insert a combination of code and text/markdown cells here to explore your data.

**Step 0: Sample data**

In [21]:
%%sql
select *
from abq
limit 3;

3 rows affected.


facility_name,facility_key,site_address,city,state,zip,owner_key,owner_name,nature_of_business,street_number,street_name,street_type,post_directional,phone,program_category,program_category_description,inspection_date,inspection_type,inspection_desc,serial_num,action_code,action_desc,result_code,result_desc,violation_code,violation_desc,inspection_memo
00BSAY IT WITH CAKE,50534,10200 GOLF COURSE RD NW STE B,ALBUQUERQUE,NM,87114,47341,OOBSAY IT WITH CAKE,CAKE DECORATING,10200,GOLF COURSE,RD,NW,5058339563,0405,Food Processor,2009-10-21 00:00:00,None,EMERGENCY RESPONSE,DA8009125,00,NON-GRADED,00,NOT APPLICABLE,68,Delivered Informational Brochure,None
00BSAY IT WITH CAKE,50534,10200 GOLF COURSE RD NW STE B,ALBUQUERQUE,NM,87114,47341,OOBSAY IT WITH CAKE,CAKE DECORATING,10200,GOLF COURSE,RD,NW,5058339563,0405,Food Processor,2009-10-21 00:00:00,None,ROUTINE FOOD INSPECTION,DA8009123,11,APPROVED,03,IN COMPLIANCE,00,No Violations Found,None
00BSAY IT WITH CAKE,50534,10200 GOLF COURSE RD NW STE B,ALBUQUERQUE,NM,87114,47341,OOBSAY IT WITH CAKE,CAKE DECORATING,10200,GOLF COURSE,RD,NW,5058339563,0405,Food Processor,2009-10-21 00:00:00,None,ROUTINE FOOD INSPECTION,DA8009123,11,APPROVED,03,IN COMPLIANCE,55,Additional Comments,None


**Step 1: Descriptive statistics**

Program category description summary

In [22]:
%%sql
select program_category_description
 , count(*) as vol
from abq
group by program_category_description
order by 1;

25 rows affected.


program_category_description,vol
Bakery,343
Bar,832
Childcare,748
Food Processor,489
Food Service Establishment,7969
Food/Drink Vending - Up to 15 Units,6
Frozen Food Truck,90
Institutional Kitchen,357
Limited Menu,288
Meat Market,121


Inspection description summary

In [23]:
%%sql
select inspection_desc
 , count(*) as vol
from abq
group by inspection_desc
order by 1;

15 rows affected.


inspection_desc,vol
ADDITIONAL SERVICE PROVIDED,439
COMPLAINT FOOD INSPECTION,1
E-MAIL,1
EMERGENCY RESPONSE,1987
FIRE CALL,5
FOLLOW-UP DOCUMENTATION RECEIVED,75
FOOD FOLLOW-UP INSPECTION,143
IMMINENT HEALTH HAZARD,36
MANAGERIAL ASSISTANCE,54
MOBILE UNIT FOOD INSPECTION,377


Result description summary

In [24]:
%%sql
select result_desc
 , count(*) as vol
from abq
group by result_desc
order by 1;

19 rows affected.


result_desc,vol
DOWNGRADE,214
EMBARGO,3
FACILITY CLOSED AT TIME OF INSPECTION,19
IMMEDIATE CLOSURE,45
IN COMPLIANCE,14389
NOT APPLICABLE,2416
NOT IN COMPLIANCE,24
NOTICE,3
OUT OF BUSINESS,142
PERMISSION TO OPERATE,1


Violation description summary

In [25]:
%%sql
select violation_desc
 , count(*) as vol
from abq
group by violation_desc
order by 1;

71 rows affected.


violation_desc,vol
Additional Comments,6353
Adulterated food,28
Barehand Contact with ready to eat food,41
"Bent, swollen containers",21
Cold Holding,201
Complaint Not Substantiated,12
Complaint Substantiated,8
Contaminated equipment,20
Cooking Temperatures,6
"Cooling, Heating Procedures",57


**Step 2: Investigative Queries**

My queries below will focus on trends and insights involving restaurants that are found to have health code violations.

Question 1: Which restaurants had the largest number of inspections that resulted in a potentially negative outcome? For these purpoes, I've defined a potentially negative outcome as a violation description not equal to the ones defiend below.

In [44]:
%%sql
select facility_name
 , count(*) as vol
from abq
where violation_desc not in ('Additional Comments', 'Complaint Not Substantiated', 'No Violations Found', 'No violations found at time of inspection.', 'None')
group by facility_name
order by 2 desc
limit 10;

10 rows affected.


facility_name,vol
CIRCLE K 8936,62
DIONS PIZZA,61
OOB MOM THAI FOOD,59
CIRCLE K 8940,59
ECHO INC,59
SMITHS FOOD AND DRUG CENTERS INC,48
99 BANH SUPERMARKET,41
MCDONALDS,33
TALIN MARKET WORLD FOOD FARE,32
EL MEZQUITE MARKET LLC,27


Question 2: What do the inspection details for DIONS PIZZA look like?. Majority of items noted are from Pre-opening food inspection.

In [45]:
%%sql
select *
from abq
where violation_desc not in ('Additional Comments', 'Complaint Not Substantiated', 'No Violations Found', 'No violations found at time of inspection.', 'None')
 and facility_name = 'DIONS PIZZA'
order by inspection_date;

61 rows affected.


facility_name,facility_key,site_address,city,state,zip,owner_key,owner_name,nature_of_business,street_number,street_name,street_type,post_directional,phone,program_category,program_category_description,inspection_date,inspection_type,inspection_desc,serial_num,action_code,action_desc,result_code,result_desc,violation_code,violation_desc,inspection_memo
DIONS PIZZA,723,4200 MONTANO RD NW,ALBUQUERQUE,NM,87120,610,PETER DEFRIES CORP,RESTAURANT,4200,MONTANO,RD,NW,5058981161,0406,Food Service Establishment,2009-11-06 00:00:00,None,EMERGENCY RESPONSE,DA8009281,00,NON-GRADED,00,NOT APPLICABLE,68,Delivered Informational Brochure,None
DIONS PIZZA,723,4200 MONTANO RD NW,ALBUQUERQUE,NM,87120,610,PETER DEFRIES CORP,RESTAURANT,4200,MONTANO,RD,NW,5058981161,0406,Food Service Establishment,2009-11-06 00:00:00,None,ROUTINE FOOD INSPECTION,DA8009279,11,APPROVED,03,IN COMPLIANCE,38,Food unprotected,None
DIONS PIZZA,90924,6308 4TH ST NW,ALBUQUERQUE,NM,87107,610,PETER DEFRIES CORP,RESTAURANT,6308,4TH,ST,NW,5053454900,0406,Food Service Establishment,2009-12-16 00:00:00,None,PRE-OPENING FOOD INSPECTION,DA0059230,11,APPROVED,03,IN COMPLIANCE,04 01,Cold Holding,None
DIONS PIZZA,90924,6308 4TH ST NW,ALBUQUERQUE,NM,87107,610,PETER DEFRIES CORP,RESTAURANT,6308,4TH,ST,NW,5053454900,0406,Food Service Establishment,2009-12-16 00:00:00,None,PRE-OPENING FOOD INSPECTION,DA0059230,11,APPROVED,03,IN COMPLIANCE,04 02,Hot Holding,None
DIONS PIZZA,90924,6308 4TH ST NW,ALBUQUERQUE,NM,87107,610,PETER DEFRIES CORP,RESTAURANT,6308,4TH,ST,NW,5053454900,0406,Food Service Establishment,2009-12-16 00:00:00,None,PRE-OPENING FOOD INSPECTION,DA0059230,11,APPROVED,03,IN COMPLIANCE,04 03,Reheating Temp,None
DIONS PIZZA,90924,6308 4TH ST NW,ALBUQUERQUE,NM,87107,610,PETER DEFRIES CORP,RESTAURANT,6308,4TH,ST,NW,5053454900,0406,Food Service Establishment,2009-12-16 00:00:00,None,PRE-OPENING FOOD INSPECTION,DA0059230,11,APPROVED,03,IN COMPLIANCE,04 04,Cooking Temperatures,None
DIONS PIZZA,90924,6308 4TH ST NW,ALBUQUERQUE,NM,87107,610,PETER DEFRIES CORP,RESTAURANT,6308,4TH,ST,NW,5053454900,0406,Food Service Establishment,2009-12-16 00:00:00,None,PRE-OPENING FOOD INSPECTION,DA0059230,11,APPROVED,03,IN COMPLIANCE,04 05,"Cooling, Heating Procedures",None
DIONS PIZZA,90924,6308 4TH ST NW,ALBUQUERQUE,NM,87107,610,PETER DEFRIES CORP,RESTAURANT,6308,4TH,ST,NW,5053454900,0406,Food Service Establishment,2009-12-16 00:00:00,None,PRE-OPENING FOOD INSPECTION,DA0059230,11,APPROVED,03,IN COMPLIANCE,04 06,Personnel w/infection restricted/excluded,None
DIONS PIZZA,90924,6308 4TH ST NW,ALBUQUERQUE,NM,87107,610,PETER DEFRIES CORP,RESTAURANT,6308,4TH,ST,NW,5053454900,0406,Food Service Establishment,2009-12-16 00:00:00,None,PRE-OPENING FOOD INSPECTION,DA0059230,11,APPROVED,03,IN COMPLIANCE,04 07,Improper handwashing,None
DIONS PIZZA,90924,6308 4TH ST NW,ALBUQUERQUE,NM,87107,610,PETER DEFRIES CORP,RESTAURANT,6308,4TH,ST,NW,5053454900,0406,Food Service Establishment,2009-12-16 00:00:00,None,PRE-OPENING FOOD INSPECTION,DA0059230,11,APPROVED,03,IN COMPLIANCE,04 08,Barehand Contact with ready to eat food,None


Question 3: Which restaurants had the largest number of emergency or health hazard inspections?

In [39]:
%%sql
select facility_name
 , count(*) as vol
from abq
where inspection_desc in ('EMERGENCY RESPONSE', 'IMMINENT HEALTH HAZARD')
group by facility_name
order by 2 desc
limit 10;

10 rows affected.


facility_name,vol
SMITHS FOOD AND DRUG CENTERS INC,15
OOB MARKET FRESH CAFE,10
SUBWAY,10
VILLA PIZZA INC,9
GRAHAM CENTRAL STATION,7
MCDONALDS,7
OOB ROSYS,7
WECKS,6
CHICHARRONERIA DON CHOCHE,6
99 BANH SUPERMARKET,6


Question 4: What do the emergency/hazard inspections look like for OOB Market fresh Cafe? It appears that all of these violations were associated with the same inspection. 

In [40]:
%%sql
select *
from abq
where inspection_desc in ('EMERGENCY RESPONSE', 'IMMINENT HEALTH HAZARD')
 and facility_name = 'OOB MARKET FRESH CAFE'
order by inspection_date desc;

10 rows affected.


facility_name,facility_key,site_address,city,state,zip,owner_key,owner_name,nature_of_business,street_number,street_name,street_type,post_directional,phone,program_category,program_category_description,inspection_date,inspection_type,inspection_desc,serial_num,action_code,action_desc,result_code,result_desc,violation_code,violation_desc,inspection_memo
OOB MARKET FRESH CAFE,85872,800 LOUISIANA BLVD NE,ALBUQUERQUE,NM,87108,77288,ZUBAIR REZA,COFFEE SHOP,800,LOUISIANA,BLVD,NE,5057152870,0406,Food Service Establishment,2009-07-23 00:00:00,None,IMMINENT HEALTH HAZARD,DA7256041,06,UNSATISFACTORY,14,IMMEDIATE CLOSURE,07,Improper handwashing,None
OOB MARKET FRESH CAFE,85872,800 LOUISIANA BLVD NE,ALBUQUERQUE,NM,87108,77288,ZUBAIR REZA,COFFEE SHOP,800,LOUISIANA,BLVD,NE,5057152870,0406,Food Service Establishment,2009-07-23 00:00:00,None,IMMINENT HEALTH HAZARD,DA7256041,06,UNSATISFACTORY,14,IMMEDIATE CLOSURE,16,"Inadequate water source, Hot or cold",None
OOB MARKET FRESH CAFE,85872,800 LOUISIANA BLVD NE,ALBUQUERQUE,NM,87108,77288,ZUBAIR REZA,COFFEE SHOP,800,LOUISIANA,BLVD,NE,5057152870,0406,Food Service Establishment,2009-07-23 00:00:00,None,IMMINENT HEALTH HAZARD,DA7256041,06,UNSATISFACTORY,14,IMMEDIATE CLOSURE,19,"Handwash Facilities - Inaccessible, unavailable",None
OOB MARKET FRESH CAFE,85872,800 LOUISIANA BLVD NE,ALBUQUERQUE,NM,87108,77288,ZUBAIR REZA,COFFEE SHOP,800,LOUISIANA,BLVD,NE,5057152870,0406,Food Service Establishment,2009-07-23 00:00:00,None,IMMINENT HEALTH HAZARD,DA7256041,06,UNSATISFACTORY,14,IMMEDIATE CLOSURE,30,Poor Hygienic Practices- No hair restraints,None
OOB MARKET FRESH CAFE,85872,800 LOUISIANA BLVD NE,ALBUQUERQUE,NM,87108,77288,ZUBAIR REZA,COFFEE SHOP,800,LOUISIANA,BLVD,NE,5057152870,0406,Food Service Establishment,2009-07-23 00:00:00,None,IMMINENT HEALTH HAZARD,DA7256041,06,UNSATISFACTORY,14,IMMEDIATE CLOSURE,36,"Thermometers not provided, not accurate/consp",None
OOB MARKET FRESH CAFE,85872,800 LOUISIANA BLVD NE,ALBUQUERQUE,NM,87108,77288,ZUBAIR REZA,COFFEE SHOP,800,LOUISIANA,BLVD,NE,5057152870,0406,Food Service Establishment,2009-07-23 00:00:00,None,IMMINENT HEALTH HAZARD,DA7256041,06,UNSATISFACTORY,14,IMMEDIATE CLOSURE,40,Food improperly labeled,None
OOB MARKET FRESH CAFE,85872,800 LOUISIANA BLVD NE,ALBUQUERQUE,NM,87108,77288,ZUBAIR REZA,COFFEE SHOP,800,LOUISIANA,BLVD,NE,5057152870,0406,Food Service Establishment,2009-07-23 00:00:00,None,IMMINENT HEALTH HAZARD,DA7256041,06,UNSATISFACTORY,14,IMMEDIATE CLOSURE,45,"Floors, Walls, Ceilings",None
OOB MARKET FRESH CAFE,85872,800 LOUISIANA BLVD NE,ALBUQUERQUE,NM,87108,77288,ZUBAIR REZA,COFFEE SHOP,800,LOUISIANA,BLVD,NE,5057152870,0406,Food Service Establishment,2009-07-23 00:00:00,None,IMMINENT HEALTH HAZARD,DA7256041,06,UNSATISFACTORY,14,IMMEDIATE CLOSURE,49,Other Operations - Linen,None
OOB MARKET FRESH CAFE,85872,800 LOUISIANA BLVD NE,ALBUQUERQUE,NM,87108,77288,ZUBAIR REZA,COFFEE SHOP,800,LOUISIANA,BLVD,NE,5057152870,0406,Food Service Establishment,2009-07-23 00:00:00,None,IMMINENT HEALTH HAZARD,DA7256041,06,UNSATISFACTORY,14,IMMEDIATE CLOSURE,52,IMMEDIATE CLOSURE - IMMINENT HEALTH HAZARD,None
OOB MARKET FRESH CAFE,85872,800 LOUISIANA BLVD NE,ALBUQUERQUE,NM,87108,77288,ZUBAIR REZA,COFFEE SHOP,800,LOUISIANA,BLVD,NE,5057152870,0406,Food Service Establishment,2009-07-23 00:00:00,None,IMMINENT HEALTH HAZARD,DA7256041,06,UNSATISFACTORY,14,IMMEDIATE CLOSURE,55,Additional Comments,None


Question 5: Based on my examination above, I will adjust my query to identify the restaurants with the largest number of emergency/health hazard violations on distinct inspection dates.

In [42]:
%%sql
select b.facility_name
 , count(b.*) as vol
from 
    (select distinct facility_name, inspection_date
    from abq
    where inspection_desc in ('EMERGENCY RESPONSE', 'IMMINENT HEALTH HAZARD')) as b
group by b.facility_name
order by count(b.*) desc
limit 10;


10 rows affected.


facility_name,vol
SUBWAY,10
SMITHS FOOD AND DRUG CENTERS INC,7
MCDONALDS,7
WECKS,6
LA PETITE ACADEMY,6
BLAKES LOTABURGER,5
DIONS,5
BURGER KING,5
KEVA JUICE SW LLC,4
GOLDEN PRIDE CHICKEN,4


Question 6: What does the inspection detail for Subway look like? It appears these pertain to different locations.

In [43]:
%%sql
select *
from abq
where inspection_desc in ('EMERGENCY RESPONSE', 'IMMINENT HEALTH HAZARD')
 and facility_name = 'SUBWAY'
order by inspection_date desc;

10 rows affected.


facility_name,facility_key,site_address,city,state,zip,owner_key,owner_name,nature_of_business,street_number,street_name,street_type,post_directional,phone,program_category,program_category_description,inspection_date,inspection_type,inspection_desc,serial_num,action_code,action_desc,result_code,result_desc,violation_code,violation_desc,inspection_memo
SUBWAY,79381,5850 EUBANK BLVD NE,ALBUQUERQUE,NM,87111,72487,DIACOUR LLC,RESTAURANT,5850,EUBANK,BLVD,NE,5052174112,0406,Food Service Establishment,2010-01-06 00:00:00,None,EMERGENCY RESPONSE,DA7507370,00,NON-GRADED,00,NOT APPLICABLE,68,Delivered Informational Brochure,None
SUBWAY,83163,8101 SAN PEDRO BLVD NE,ALBUQUERQUE,NM,87113,68797,SUBWAY,SANDWICH SHOP,8101,SAN PEDRO,BLVD,NE,5057972298,0406,Food Service Establishment,2009-12-22 00:00:00,None,EMERGENCY RESPONSE,DA7507245,00,NON-GRADED,00,NOT APPLICABLE,68,Delivered Informational Brochure,None
SUBWAY,88734,8000 ACADEMY RD NE,ALBUQUERQUE,NM,87111,80562,WIGH LLC,DELI,8000,ACADEMY,RD,NE,5058220074,0406,Food Service Establishment,2009-12-15 00:00:00,None,EMERGENCY RESPONSE,DA7507180,00,NON-GRADED,00,NOT APPLICABLE,68,Delivered Informational Brochure,None
SUBWAY,28313,3200 COORS BLVD NW,ALBUQUERQUE,NM,87120,26318,JUAN C. LEVY,FOOD SERVICE,3200,COORS,BLVD,NW,5053520094,0406,Food Service Establishment,2009-11-30 00:00:00,None,EMERGENCY RESPONSE,DA8009384,00,NON-GRADED,00,NOT APPLICABLE,68,Delivered Informational Brochure,None
SUBWAY,71764,2225K WYOMING BLVD NE,ALBUQUERQUE,NM,87112,64201,ARYAVART INC,SANDWICHES,2225K,WYOMING,BLVD,NE,5052751548,0406,Food Service Establishment,2009-11-05 00:00:00,None,EMERGENCY RESPONSE,DA7756789,00,NON-GRADED,00,NOT APPLICABLE,68,Delivered Informational Brochure,None
SUBWAY,28312,11111 MENAUL BLVD NE,ALBUQUERQUE,NM,87112,26318,JUAN C. LEVY,FOOD SERVICE,11111,MENAUL,BLVD,NE,5052966783,0406,Food Service Establishment,2009-10-30 00:00:00,None,EMERGENCY RESPONSE,DA7756731,00,NON-GRADED,00,NOT APPLICABLE,68,Delivered Informational Brochure,None
SUBWAY,71374,523 LOMAS BLVD NW,ALBUQUERQUE,NM,87102,46949,G & P INVESTMENTS INC,RESTUARANT,523,LOMAS,BLVD,NW,5052430249,0406,Food Service Establishment,2009-10-26 00:00:00,None,EMERGENCY RESPONSE,DA8508436,00,NON-GRADED,00,NOT APPLICABLE,None,None,None
SUBWAY,28313,3200 COORS BLVD NW,ALBUQUERQUE,NM,87120,26318,JUAN C. LEVY,FOOD SERVICE,3200,COORS,BLVD,NW,5053520094,0406,Food Service Establishment,2009-09-25 00:00:00,None,EMERGENCY RESPONSE,DA8008894,00,NON-GRADED,00,NOT APPLICABLE,68,Delivered Informational Brochure,None
SUBWAY,79380,200 3RD ST NW STE 250,ALBUQUERQUE,NM,87120,72487,DIACOUR LLC,RESTAURANT,200,3RD,ST,NW,5052449636,0406,Food Service Establishment,2009-09-11 00:00:00,None,EMERGENCY RESPONSE,DA7005841,00,NON-GRADED,18,TRAINING RECEIVED,55,Additional Comments,None
SUBWAY,13328,640 COORS BLVD NW,ALBUQUERQUE,NM,87105,11977,BUCKMAR CORP INC,RESTAURANT,640,COORS,BLVD,NW,5058311588,0406,Food Service Establishment,2009-08-12 00:00:00,None,EMERGENCY RESPONSE,DA0040973,00,NON-GRADED,00,NOT APPLICABLE,None,None,None


Question 7: Based on my examination above, I will adjust my query to identify the restaurants with the largest number of emergency/health hazard violations on distinct inspection dates for a distinct restaurant key.

In [46]:
%%sql
select b.facility_name
 , b.facility_key
 , count(b.*) as vol
from 
    (select distinct facility_name, facility_key, inspection_date
    from abq
    where inspection_desc in ('EMERGENCY RESPONSE', 'IMMINENT HEALTH HAZARD')) as b
group by b.facility_name
 , b.facility_key
order by count(b.*) desc
limit 10;

10 rows affected.


facility_name,facility_key,vol
GLORIAS NM BURRITOS AND MORE,87553,2
OOB PER WRITTEN REQUEST K MART,2608,2
CIBOLA -DECA SNACK BAR,71571,2
PER EHSII ROSANNA TRUJILLO FA OOB 02/20/14 VALERO,12288,2
KEVA JUICE SW LLC,73182,2
OOB CHURCHILL CHEVRON REDIMART,16260,2
HECTORS BAKERY,31322,2
TARGET STORE 356,8881,2
LOWELL CHILD DEVELOPMENT CENTER,81272,2
CIBOLA HIGH SCHOOL,2140,2


Based on above, there are not too many repeat offenders, as no restaurant had more than 2 distinct emergency/health harzard inspections. Below is the same query with my original definition of a potentially negative inspection from Question 1.

In [49]:
%%sql
select b.facility_name
 , b.facility_key
 , count(b.*) as vol
from 
    (select distinct facility_name, facility_key, inspection_date
    from abq
    where violation_desc not in ('Additional Comments', 'Complaint Not Substantiated', 'No Violations Found', 'No violations found at time of inspection.', 'None')) as b
group by b.facility_name
 , b.facility_key
order by count(b.*) desc
limit 10;

10 rows affected.


facility_name,facility_key,vol
99 BANH SUPERMARKET,18468,5
EL MEZQUITE MARKET LLC,60817,4
OOB POP N TACO DRIVE UP INC.,48566,4
OOB GARDUNOS CHILE PACKING CO,5871,4
CHEVRON,87139,4
LA MONTANITA NATL. FOOD COOP.,4068,3
WALGREENS 03997,1991,3
OOB BLACK MESA COFFEE CO,20973,3
OOB CAPTAIN DS SEAFOOD,6502,3
KEVA JUICE SW LLC,73182,3


Finally, the negative inspection detail for 99 Banh Supermarket which based on my analysis is not a place where I would shop!

In [50]:
%%sql
select *
from abq
where facility_key = '18468'
and violation_desc not in ('Additional Comments', 'Complaint Not Substantiated', 'No Violations Found', 'No violations found at time of inspection.', 'None')
order by inspection_date;

41 rows affected.


facility_name,facility_key,site_address,city,state,zip,owner_key,owner_name,nature_of_business,street_number,street_name,street_type,post_directional,phone,program_category,program_category_description,inspection_date,inspection_type,inspection_desc,serial_num,action_code,action_desc,result_code,result_desc,violation_code,violation_desc,inspection_memo
99 BANH SUPERMARKET,18468,5315 GIBSON BLVD SE,ALBUQUERQUE,NM,87108,16883,GOLDEN CITIES INC,GROCERY STORE,5315,GIBSON,BLVD,SE,5052682422,0601,Retail -Grocery,2009-07-21 00:00:00,None,ROUTINE FOOD INSPECTION,DA7256034,11,APPROVED,03,IN COMPLIANCE,38,Food unprotected,None
99 BANH SUPERMARKET,18468,5315 GIBSON BLVD SE,ALBUQUERQUE,NM,87108,16883,GOLDEN CITIES INC,GROCERY STORE,5315,GIBSON,BLVD,SE,5052682422,0601,Retail -Grocery,2009-07-21 00:00:00,None,ROUTINE FOOD INSPECTION,DA7256034,11,APPROVED,03,IN COMPLIANCE,40,Food improperly labeled,None
99 BANH SUPERMARKET,18468,5315 GIBSON BLVD SE,ALBUQUERQUE,NM,87108,16883,GOLDEN CITIES INC,GROCERY STORE,5315,GIBSON,BLVD,SE,5052682422,0608,Retail - Deli,2009-07-21 00:00:00,None,ROUTINE FOOD INSPECTION,DA7256035,11,APPROVED,03,IN COMPLIANCE,34,Food contact surfaces,None
99 BANH SUPERMARKET,18468,5315 GIBSON BLVD SE,ALBUQUERQUE,NM,87108,16883,GOLDEN CITIES INC,GROCERY STORE,5315,GIBSON,BLVD,SE,5052682422,0608,Retail - Deli,2009-07-21 00:00:00,None,ROUTINE FOOD INSPECTION,DA7256035,11,APPROVED,03,IN COMPLIANCE,32,Improper storage/handling of clean equipment/utens,None
99 BANH SUPERMARKET,18468,5315 GIBSON BLVD SE,ALBUQUERQUE,NM,87108,16883,GOLDEN CITIES INC,GROCERY STORE,5315,GIBSON,BLVD,SE,5052682422,0608,Retail - Deli,2009-07-21 00:00:00,None,ROUTINE FOOD INSPECTION,DA7256035,11,APPROVED,03,IN COMPLIANCE,02,Hot Holding,None
99 BANH SUPERMARKET,18468,5315 GIBSON BLVD SE,ALBUQUERQUE,NM,87108,16883,GOLDEN CITIES INC,GROCERY STORE,5315,GIBSON,BLVD,SE,5052682422,0608,Retail - Deli,2009-07-21 00:00:00,None,ROUTINE FOOD INSPECTION,DA7256035,11,APPROVED,03,IN COMPLIANCE,45,"Floors, Walls, Ceilings",None
99 BANH SUPERMARKET,18468,5315 GIBSON BLVD SE,ALBUQUERQUE,NM,87108,16883,GOLDEN CITIES INC,GROCERY STORE,5315,GIBSON,BLVD,SE,5052682422,0608,Retail - Deli,2009-07-21 00:00:00,None,ROUTINE FOOD INSPECTION,DA7256035,11,APPROVED,03,IN COMPLIANCE,12,Enough facilities to maintain product temps,None
99 BANH SUPERMARKET,18468,5315 GIBSON BLVD SE,ALBUQUERQUE,NM,87108,16883,GOLDEN CITIES INC,GROCERY STORE,5315,GIBSON,BLVD,SE,5052682422,0608,Retail - Deli,2009-07-21 00:00:00,None,ROUTINE FOOD INSPECTION,DA7256035,11,APPROVED,03,IN COMPLIANCE,42,Plumbing: Improperly installed/maintained/supplied,None
99 BANH SUPERMARKET,18468,5315 GIBSON BLVD SE,ALBUQUERQUE,NM,87108,16883,GOLDEN CITIES INC,GROCERY STORE,5315,GIBSON,BLVD,SE,5052682422,0608,Retail - Deli,2009-07-21 00:00:00,None,ROUTINE FOOD INSPECTION,DA7256035,11,APPROVED,03,IN COMPLIANCE,35,Non-Food contact surfaces,None
99 BANH SUPERMARKET,18468,5315 GIBSON BLVD SE,ALBUQUERQUE,NM,87108,16883,GOLDEN CITIES INC,GROCERY STORE,5315,GIBSON,BLVD,SE,5052682422,0601,Retail -Grocery,2009-08-17 00:00:00,None,ROUTINE FOOD INSPECTION,DA7256128,11,APPROVED,03,IN COMPLIANCE,38,Food unprotected,None


## Bonus (10 points)

We have seen a few simple techniques for normalizing data in class.  Choose a column or set of columns that might be redundant and normalize them out to another table:

* Identify the columns to normalize in the source data table
* Create one or more new tables to hold the normalized records
* Extract data into the new tables
* Re-insert the new foreign key references from the new table into the source table

Did not have time to complete bonus portion of assignment.